<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Sentence-embedding" data-toc-modified-id="Sentence-embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sentence embedding</a></span><ul class="toc-item"><li><span><a href="#Infersent" data-toc-modified-id="Infersent-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Infersent</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#PCA" data-toc-modified-id="PCA-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>PCA</a></span></li></ul></li><li><span><a href="#Blocking" data-toc-modified-id="Blocking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Blocking</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Searching" data-toc-modified-id="Searching-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Searching</a></span></li></ul></li><li><span><a href="#Result" data-toc-modified-id="Result-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Result</a></span></li></ul></div>

# Setup 

In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from preprocessing_datasets.preprocessing_utilities import get_labels_by

params = {}

# Import dataset

In [2]:
from preprocessing_datasets.preprocessing_census import clean_census
table, pairs = clean_census()

In [3]:
table

,last_name,first_name,middle_name,zip_code,street_address
0,anderson,unk,unk,4848,basswood
1,anderson,unk,unk,4848,basswood
2,anderson,unk,unk,4848,basswood
3,anderson,unk,unk,4848,basswood
4,aquendo,clara,j,666,starkey
...,...,...,...,...,...
836,william,sherry,v,510,woodhaven
837,wright,bryan,unk,307,woodhaven
838,wright,maxine,h,307,woodhaven
839,yates,chanse,e,403,woodhaven


# Sentence embedding

## Infersent

Setup model

In [4]:
from embedding_algorithms.inferSent import set_RNN_embedding
start = time.time()
model_type = "bilstm" 
char_level = False
model_version = 2
rnn_dim = 2048
verbose = 1
set_RNN_embedding(model_type, char_level, model_version, rnn_dim, verbose)
print("TIME: {0}".format(time.time() - start))

params['embedding'] = {
    'name': 'Infersent',
    'model_type': model_type,
    'char_level': char_level,
    'model_version': model_version,
    'rnn_dim': rnn_dim,
    'verbose': verbose
}

Vocab size : 2196017
TIME: 186.05267095565796


Embedding

In [5]:
from embedding_algorithms.inferSent import RNN_embedding
attr_list = ['last_name', 'first_name', 'zip_code', 'street_address']
params['attr_list'] = attr_list
embeddings_tokens = RNN_embedding(table, attr_list, model_type, char_level)
embeddings_tokens = np.array(embeddings_tokens)

attrs: ['last_name', 'first_name', 'zip_code', 'street_address']


# Dimensionality reduction

## PCA

In [6]:
from dimensionality_reduction_algorithms.pca import pca_dim_reduction

start = time.time()
pca_embeddings = pca_dim_reduction(
            embeddings_tokens, 
            num_components=2,
            verbose=1)

print("PCA: {0}".format(time.time() - start))

params['reduction'] = {
    'name': 'PCA',
    'num_components': 2,
    'verbose': 1,
}

starting dimension: 4096
setting PCA with n_components: 2
PCA: 0.16042208671569824


# Blocking

In [11]:
from cluster_algorithms.hierarchy_cluster import hierarchy_cluster
num_clusters = 3
start = time.time()

blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
print("BLOCKS: {0}".format(time.time() - start))

params['blocking'] = {
    'name': 'hierarchy',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 3, 
BLOCKS: 0.024881839752197266


# Evaluation

In [12]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.5646282769945077
(PC) Pair completeness is: 0.8197674418604651
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6686872430040482
(PQ) Pair quality - Precision is: 0.0018337646798715064
(FM) Fmeasure is: 0.0036593436538935676

['last_name', 'first_name', 'zip_code', 'street_address']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'PCA', 'num_components': 2, 'verbose': 1}
{'name': 'hierarchy', 'num_clusters': 3}


## Searching

In [10]:
for i in range(1,15,1):
    num_clusters = int(i)
    blocks = hierarchy_cluster(pca_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

clustering with NUM_CLUSTERS = 1, 
(RR) Reduction ratio is: 0.0
(PC) Pair completeness is: 1.0
(RM) Reference metric (Harmonic mean RR and PC) is: 0
(PQ) Pair quality - Precision is: 0.0009738972878092973
(FM) Fmeasure is: 0.0019458994694029935
clustering with NUM_CLUSTERS = 2, 
(RR) Reduction ratio is: 0.4827019987543174
(PC) Pair completeness is: 0.8197674418604651
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6076202179653183
(PQ) Pair quality - Precision is: 0.0015433450087565675
(FM) Fmeasure is: 0.003080889743477691
clustering with NUM_CLUSTERS = 3, 
(RR) Reduction ratio is: 0.5646282769945077
(PC) Pair completeness is: 0.8197674418604651
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6686872430040482
(PQ) Pair quality - Precision is: 0.0018337646798715064
(FM) Fmeasure is: 0.0036593436538935676
clustering with NUM_CLUSTERS = 4, 
(RR) Reduction ratio is: 0.6558575392106902
(PC) Pair completeness is: 0.7151162790697675
(RM) Reference metric (Harmonic mean RR and PC)

# Result

In [13]:
# Census
# RR PC alpha

list_hierarchy_tsne = [0.9006, 0.9622, 0.9304] 
list_hierarchy_pca = [0.5646, 0.8197, 0.6686] # PCA: 0.16, BLOCKS: 0.024, n_cluster: 3
